In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


## Bellman-Ford (Cont'd)

### Computing $\delta^{k+1}(u,v)$

As usual, we will assume we have recursively computed $\delta^k(s,v)$. To extend this to compute $\delta^{k+1}(s,v)$:


$$
\begin{align}
\delta^{k+1}(s, v) = \min&(\delta^{k}(s, v),\\
&\min_{x \in N^-(v)} (\delta^{k}(s, x) + w(x,v))
\end{align}
$$
where $N^-(v)$ are the *in-neighbors* of $v$

<br><br>


### Illustration Example

<center>
<img src="figures/bellman2.gif" width=50%/>
</center>






In [5]:
import math


# represent in-neighbors for each node, for constant lookup
graph = {
            's': {},
            'a': {('s', 5), ('c', 2)},
            'b': {('a', 1), ('c', 4)},
            'c': {('s', -2)}, 
            'd': {('b', -1), ('c', 4)},
            't': {('b', 3), ('d', 1)},
        }


def bellmanford(graph, source):
    def bellmanford_helper(distances, k):        
        if k == len(graph): # negative cycle
            return -math.inf
        else:
            # compute new distances
            new_distances = compute_distances(graph, distances)
            
            # check if distances have converged
            if converged(distances, new_distances):
                return distances
            else:                
                return bellmanford_helper(new_distances, k+1) ## increase level by 1 [BFS]
        
    # initialize
    distances = dict()
    for v in graph:
        if v == source:
            distances[v] = 0
        else:
            distances[v] = math.inf
    return bellmanford_helper(distances, 0) ## start from source

def compute_distances(graph, distances):
    new_distances = {}
    for v, in_neighbors in graph.items(): # this loop can be done in parallel
        # compute all possible distances from s->v
        v_distances = [distances[v]] 
        for in_neighbor, weight in in_neighbors:
            v_distances.append(distances[in_neighbor] + weight)
        new_distances[v] = min(v_distances)
    return new_distances

def converged(old_distances, new_distances):
    for k in old_distances:
        if old_distances[k] != new_distances[k]:
            return False
    return True


bellmanford(graph, 's')

{'s': 0, 'a': 0, 'b': 1, 'c': -2, 'd': 0, 't': 1}

### Cost of Bellman-Ford

```python
def compute_distances(graph, distances):
    new_distances = {}
    for v, in_neighbors in graph.items(): # this loop can be done in parallel
        v_distances = [distances[v]]
        for in_neighbor, weight in in_neighbors:
            v_distances.append(distances[in_neighbor] + weight)
        new_distances[v] = min(v_distances)
    return new_distances
```

- For each vertex, we loop through all of its in-neighbors.
- We then take the minimum over its in-neighbors
- Thus, we will visit each edge in the graph once in each iteration of the algorithm ($|E|$)
- There are at most $|V|$ iterations of the algorithm, to the **total work** for `compute_distances` is $O(|V| \cdot |E|)$.

Because we can do the outer loop in parallel, we must consider the maximum work of any vertex $v$.

In the worst case, $v$ can have $|V|-1$ in-neighbors. The `min` operation will then take $O(\lg |V|)$ span, assuming we use `reduce` to implement it.

Thus, each iteration has $O(\lg |V|)$ span, and we have at worst $|V|$ iterations, run sequentially, resulting in **total span** of $O(|V| \lg |V|)$.



<br><br>
What about the work to check if we have converged?

```python
def converged(old_distances, new_distances):
    for k in old_distances:
        if old_distances[k] != new_distances[k]:
            return False
    return True
```

Actually, if we were more clever, we could include this check inside the `compute_distances` function, just before assigning the `new_distances` value.

```python
...
min_v = min(v_distances)
if min_v != new_distances[v]:
    converged=False
new_distances[v] = min(v_distances)
...
```
so, we don't incur any additional cost for this check.


<br>

Thus, the final work/span of Bellman-Ford is:

**Work:** $O(|V| \cdot |E|)$

**Span:** $O(|V| \lg |V|)$


<br>

compare with:

- Dijkstra: Work$=O(|E|\log |E|)$, Span$=O(|E|\log |E|)$ [`min-heap for insertion and deletion`]
- BFS: Work$=O(|E|+ |V|)$, Span$=O(|E| + |V|)$


So, we can see we pay significant costs going from unweighted $\rightarrow$ weighted-positive $\rightarrow$ weighted-negative graphs.

- Although, consider the difference in span between Dijkstra and Bellman-Ford